# Intro to Text as Data (UMN LATIS/Libraries)

## Intro
This workshop will cover how to:
- Read and write text files in Python
- Manipulate ‘strings’ of text
- Pre-process and clean text for analysis
- Count and plot word frequencies
- Conduct sentiment analysis
- ~Run basic topic models~

### Prereqs
Be familiar with Python. Have some previous experience with an introductory Python workshop, for example.

## JupyterLab - Get attendees set up
We'll share JupyterLab features as we go, but to get started let's create a new blank `workshop.ipynb` file that you can work with throughout the workshop. 

Note the control icons above the Notebook. You can use these to:
- add new cells
- run code (which you can also do, using control-shift)
- and switch cells from code to markdown

### Reading files

Let's start to work with text by reading it in from a series of text files.

For most text as data projects, your first step is going to be to read in the files containing the data. Common file types for text data are: 
* `.txt`
* `.csv`
* `.json`
* `.html` 
* `.xml`

Each file format requires specific Python tools or methods to read, but for our case, we'll be working with .txt files.

#### Reading in `.txt` files

Python has built-in support for reading in `.txt` files.

Let's take a look at the first file in our directory (folder) of State of the Union addresses which is located one directory back from the `text_as_data` folder (`../sotu_kaggle`). These text files are available to [download on Kaggle](https://www.kaggle.com/datasets/rtatman/state-of-the-union-corpus-1989-2017).

In [ ]:
# create a new variable called file1 and read ("r") the first file in the sotu_text folder
file1 = open('../sotu_kaggle/Clinton_1995.txt','r') 

If we try to print out the file, however, it's not yet stored as a Python string, but as an encoded text "wrapper" from the io (input/output) Python module.

In [ ]:
# Printing doesn't print the contents
print(file1)

To access the text, we can use the .read() method of the `TextIOWrapper` and then use the string index of the text object to show the first 250 characters. Once we save the contents of the file1 text to a new variable we can close the file1 object so that our computer memory doesn't have to keep track of that.

In [ ]:
text = file1.read()
print(text[0:250])
file1.close()

A more efficient way to work with text files in Python is to use a `with` statement, which will automatically close the file for you:

In [ ]:
with open('../sotu_kaggle/Clinton_1995.txt','r') as file1:
    text = file1.read()   

### Tokenization

Once we've read in the data, a common next step is to split a longer string into words or-—more helpfully-—"word chunks" called "tokens." This step is referred to as "tokenization". Tokens can be of any length (sentences, words, or parts of words), but usually we want to reduce our text into meaningful word chunks. 

#### Tokenizing by whitespace
An easy way to tokenize a string, though not the most accurate, is to split up a string using whitespace. Let's save each word to a list variable called 'tokens.' `.split()` creates a Python list of the words from the string. If we leave the argument for `split()` blank, it will split on whitespace characters in the string.

In [ ]:
tokens = text.split()

In [ ]:
tokens[0:10]

### Splitting on other characters

Sentence segmentation involves identifying the boundaries of sentences, and provides a different way to tokenize our text.

#### Sentence segmentation by splitting on punctuation

In [ ]:
# instead of the default whitespace for split(), you can identify the character or characters you'd like to split on
sentences = text.split('.')
sentences[0]

We can check how many items are in any list using the len() function.

In [ ]:
len(sentences)

In [ ]:
# note that this method doesn't break out sentences that end with other punctuation, like question marks
sentences[40]

### Counting words

Once we have a list of terms, we can count them up using a Python module called "Counter" in the collections package.

In [ ]:
#import the Counter module from the collections package
from collections import Counter

token_counts = Counter(tokens)

#print(token_counts)

If you hit tab after ```token_counts.``` you can see Counter methods that are available to examine this data more closely. We can use ```.most_common(5)``` to look at the five most common words in the corpus. (You can change the argument to print out as many of the most common words as you want)

In [ ]:
token_counts.most_common(5)

You can also look at specific word counts by calling the term in the same way you would refer to a Python dictionary key:

In [ ]:
token_counts['President']

We can improve this token list for later analysis by using built-in string methods to clean the data a bit. We'll create a new list, called "unigrams," to hold each token. Unigrams refer to single words, while "bigrams" are two word pairs, and "trigrams" three word pairs. Researchers often refer to these collections of tokens of different lengths as "ngrams."

We'll use the built-in string method `.lower()` to convert the text all to lower case. This helps us count up word frequencies regardless of capitalization. 

The `.replace()` method will replace specific characters in the first argument with the second argument. `'text'.replace('t', '')` for example, would change the string 'text' to 'ex', since it would replace every 't' character with an empty string.

In [ ]:
#create a new empty list
unigrams = []

# Loop through each token in the tokens list
for token in tokens:
    token = token.lower() # lowercase tokens
    token = token.replace('.', '') # remove periods
    token = token.replace('!', '') # remove exclamation points
    token = token.replace('?', '') # remove question marks
    unigrams.append(token) #append each updated token to the new unigrams list

Let's convert this list of unigrams using Counter and see if there were any changes in the most common five words.

In [ ]:
word_counts = Counter(unigrams)
print('Original list:', token_counts.most_common(5))
print('Cleaned up list:', word_counts.most_common(5))

Why do you think the counts have changed?

### Tokenizing with the Natural Language Toolkit (NLTK)* *

The steps we took above are pretty time-consuming, and require us to identify each specific character to remove. Most researchers will use the Natural Language Toolkit (NLTK) or Spacy to accomplish many of the steps we showed manually above with fewer steps. 

We can use word_tokenize tool to do a lot of the tokenizing work for us.

In [ ]:
import nltk
nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize


Let's convert our original sotu string, called ```text```, to lowercase, and tokenize it using `word_tokenize()`.

In [ ]:
word_tokens = word_tokenize(text.lower())
print(word_tokens[0:25])

#### Removing punctuation

In the example above, we have a lot of punctuation marks as tokens in our data. 

We can manually remove punctuation from strings, which can be useful in certain settings. Let's import the `string` library which includes a quick dictionary of common punctuation marks. 

In [ ]:
import string
string.punctuation

In [ ]:
# strip() will remove punctuation from the beginning or end of the string
"?Test 1, 2, 3!".strip(string.punctuation)

To remove all of the punctuation in a string we can loop through the every character in the string and use the ```.join()``` method to recombine them with an empty string. First let's see how ```.join()``` works on a simpler example:

In [ ]:
'x'.join('Put an x in between every character')

In [ ]:
# let's remove all punctuation from our SOTU speech
clean_text = ''.join(char.strip(string.punctuation) for char in text)

### Tokenize with word_tokenize
Now that we removed the punctuation from our text, we can use retokenize the corpus using `clean_text`.

In [ ]:
tokens = word_tokenize(clean_text.lower())
tokens[:10]

Now that we have a list of tokens we can count their frequencies in the speech. Let's use a builtin NLTK function called `FreqDist()` to look at our most common words. This is similar to the `Counter` library we worked with earlier, though provides some extended functionality.

In [ ]:
from nltk.probability import FreqDist

In [ ]:
#apply the FreqDist function to our tokens variable
fdist = FreqDist(tokens)

#fdist is a dictionary of unique words and the number of times they occur
fdist

In [ ]:
#fdist also includes a handy method to find the most common words 
fdist.most_common(10)

...which, to make things even more complicated, returns a list (see the square brackets continaing comma-separated items) containing tuples (those objects in parentheses, also containing comma-separated items). But we needn't get overly worried about that here. 

#### Removing stop words

You might have noticed that the most common words above aren't terribly exciting. They're words like "am", "i", "the" and "a": stop words. These are rarely useful to us in computational text analysis, so it's very common to remove them completely.

NLTK includes a stopwords module we can use. Not all stopwords lists are equal though: for your own research you might want to customize a stopwords list, or find one that is best-suited to your domain.

In [ ]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')

# how many stopwords are on the list?
len(stop)

In [ ]:
# what are the first ten word on the stopword list?
stop[0:10]

Let's create a new list of tokens, removing our stopwords along the way. 

This loop checks each word in our original tokens list, and if it does *not* appear on the stopword list, it adds it to a new list called tokens_clean.

In [ ]:
tokens_clean = [] 
  
for w in tokens: 
    if w not in stop: 
        tokens_clean.append(w)
tokens_clean[0:10]

In [ ]:
tokens[0:10]

In [ ]:
# advanced we can do the same thing quite efficiently with a list comprehension
tokens_clean = [w for w in tokens if w not in stop]
tokens_clean[0:10]

In [ ]:
# now we can re-count the most common words after stop words are removed
freq = FreqDist(tokens_clean)
freq.most_common(10)

Hmmm, still not terribly interesting but getting better...

#### Stemming

Stemming and lemmatization both refer to remove morphological affixes on words. For example, if we stem the word "grows", we get "grow". If we stem the word "running", we get "run". We do this because often we care more about the core content of the word (i.e. that it has something to do with growth or running, rather than the fact that it's a third person present tense verb, or progressive participle).

NLTK provides many algorithms for stemming. For English, a great baseline is the [Porter](https://github.com/nltk/nltk/blob/develop/nltk/stem/porter.py) algorithm.

In [ ]:
# import the PorterStemmer and then stem the word "states" as an example
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
stemmer.stem('states')

In [ ]:
stemmer.stem('government')

In [ ]:
stemmer.stem('roosevelt')

In a similar manner as the stopwords loop above, we can create a new list of stemmed tokens:

In [ ]:
tokens_stemmed = []
for t in tokens_clean:
    tokens_stemmed.append(stemmer.stem(t))

In [ ]:
#or as a comprehension:
tokens_stemmed = [stemmer.stem(t) for t in tokens_clean]

In [ ]:
tokens_stemmed[0:10]

Now that the words are stemmed, are the most common words any different? 

Here are the stemmed top ten.

In [ ]:
freq_stemmed = FreqDist(tokens_stemmed)
for f in freq_stemmed.most_common(10):
    print(f)

And the unstemmed top ten:

In [ ]:
for f in freq.most_common(10):
    print(f)

Similar, but with some important differences. Notice that "work" went from 40 to 66 after stemming.  

Why would that be?

### Reading in multiple files

Often, our text data is split across multiple files in a folder. We can read them all into a single variable using a Python tool called glob.

In [ ]:
import glob

In [ ]:
# save all of the files that end with .txt in the sotu_text/ folder to a variable called sotu_all
sotu_all = glob.glob("../sotu_kaggle/*.txt")

In [ ]:
# this just saves the file-paths to a list though
sotu_all[0:10]

Those are out of order though. Let's sort the list so that the list index is in the same order as the speeches themselves (sotu_all[0] would equal 001.txt).

*Something important to note is that **glob** can pull files differently on different systems (Windows/Mac OS/Linux). If you have a numeric identifier to your files, sorting them is always a good idea for reproducibility of your code, regardless of what system it may be run on*

In [ ]:
sotu_all.sort()

In [ ]:
sotu_all[0:10]

Note that they're in order by filename, so they're in alphabetical order. But the file names include both the President name and year. If we save the filenames as the key for a Python dictionary we can convert them to separate year and name fields later on.

Now that we have a list of all the files we need to cycle through each one and save the text from the file. To do that we'll create a new dictionary variable, `speeches`. For each file in the `sotu_all` variable we'll first clean the file name to retain the `Name_YYYY` string to the dict key, and then open and read the file to save as the dict value.

In [ ]:
speeches = {}
for speech in sotu_all:
    filename = speech.lstrip('../sotu_kaggle/')[:-4]
    with open(speech,'r') as sotu:
        speeches[filename] = sotu.read()

#### Python dictionaries

Note that we can't look at an item in a Python dict using an index number:

In [ ]:
speeches[0]

We have to use a specific key from the dict to examine the value of an item in a Python dictionary. You can views the keys using `.keys()` and values using `.values()`. You can also loop through both using the `.items()` method, which we'll do a little later on.

In [ ]:
speeches.keys()

In [ ]:
# now we can refer to each speech from the list using keys
speeches['Carter_1981'][0:250]

In [ ]:
len(speeches)

### Creating a Cleaning Function*
Now that we have all the text data loaded, we can think about working on the corpus as a whole.

Let's create a function that combines all of our cleaning protocols so that we can clean each State of the Union speech with a single piece of code. 

The function definition opens with the keyword ```def``` followed by the name of the function (clean_speeches) and a parenthesized list of parameter names (speeches). The body of the function — the statements that are executed when it runs — is indented below the definition line. The body concludes with a return keyword followed by the return value.

In [ ]:
def clean_speeches(speeches):
    #create an empty dict to hold our cleaned content
    cleaned_speeches = {}
    #loop through the dict items
    for key, speech in speeches.items():
        speech = ''.join(word.strip(string.punctuation) for word in speech.lower())
        speech = [stemmer.stem(w) for w in word_tokenize(speech) if w not in stop]
        #assign the same key to the cleaned_speeches dict, with a value of speech (our cleaned speec)
        cleaned_speeches[key] = speech
    return cleaned_speeches

You can call the function using the name of the function, and the variable you'd like to process as its parameter. We'll process our speeches list:

In [ ]:
# this cell might take a few minutes to run!
cleaned_speeches = clean_speeches(speeches)

In [ ]:
len(cleaned_speeches)

Let's look at the first ten tokens from the cleaned Carter speech.

In [ ]:
cleaned_speeches['Carter_1981'][0:10]

### Word frequencies across the corpus


#### Convert to a dataframe
Let's convert the dictionary to a dataframe so we can parse the years and names from the filenames, which will allow us to sort and plot our data by year.

In [ ]:
# convert the dictionary to a df
import pandas as pd
df = pd.DataFrame(cleaned_speeches.items(), columns=['name_year', 'text'])   
df.head()

We can create two new columns to hold the output of the `split()` method applied on the underscore symbol since that conssitently separates our name and year values.

In [ ]:
# split name_date column by delimiter
df[['name','year']] = df['name_year'].str.split('_',expand=True)
# drop the now duplicative name_year column
df = df.drop('name_year', axis=1)
df.head()

Now we can sort our dataframe by year to help us plot concepts over time. 

In [ ]:
df = df.set_index('year')
df.sort_index(inplace=True)
df

In [ ]:
#Let's drop the empty first row
df = df.drop(index='1790')

#### Add Frequencies

In [ ]:
freqs = [FreqDist(speech) for speech in df['text']]  

In [ ]:
print(freqs[0].N())
print(freqs[0]['war'])
print(freqs[0]['war']/freqs[0].N())

`FreqDist.N()` gives us the total 'outcomes', in other words the total number of words from each document. We can use this to calculate the relative frequency of a specific term to each document and track that value over time. Let's assign the contents of the `freqs` list to a new column in our df.

In [ ]:
df['freq_dist'] = freqs

In [ ]:
df.head()

We can look at the most common words by year now. 

In [ ]:
for i, row in df.iterrows():
    print(i, row['name'])
    print(row['freq_dist'].most_common(4))
    print()

Let's edit the loop above to look at the token 'war' across the full corpus and plot its frequency in each speech. We'll create three new columns:
- `word_count` pulls the word count of 'war' from each `freq_dist` object
- `total_words` uses `.N()` to add up the number of words from the `freq_dist` object (which is the cleaned text, not total word count from the speech)
- `ratio` represents the ratio of the word 'war' to the total number of words per speech

In [ ]:
word = 'war'
df['word_count'] = [freq_dist[word] for freq_dist in df['freq_dist']]
df['total_words'] = [words.N() for words in df['freq_dist']]
df['ratio'] = df['word_count'] / df['total_words']
df.head()

We won't get into much detail about matplotlib, but we can visualize elements from our dataframe using plt. 

In [ ]:
import matplotlib.pyplot as plt

# create a line plot with the values of our word_count column
ax = df['word_count'].plot(kind='line', figsize=(10, 6), title=f'Word count of "{word}"')
ax.set_xlabel('Year')
ax.set_ylabel('Count')
# only show x axis label for every tenth year
ax.set_xticks(range(0, len(df.index), 10))
ax.set_xticklabels(df.index[::10], rotation=45)
plt.show()

### Plotly
Let's use an interactive plotting library, Plotly, to build a plot that will allow us to hover over specific points to find more information. This will allow us to pinpoint the spikes in the plot that show the highest frequency of 'war' mentions.

In [ ]:
#!conda install plotly
import plotly.express as px

In [ ]:
# create a line plot assigning the x and y axes
fig = px.line(x=df.index, y=df['ratio'])
# add labels
fig.update_layout(xaxis_title="Speech",
    yaxis_title="Relative frequency of 'war'")
fig.show()

The 1944 speech has the most frequent mention of the word 'war'. Let's take a closer look.

In [ ]:
print(df.loc['1944']['name'])
print(df.loc['1944']['freq_dist'].most_common(25))


Alternately we could look at the plain word counts of 'war' from each speech. While the results are similar, it removes some spikes from speeches in 1813 and 1917, which shows that 'war' was a frequent topic relative to the length of those speeches. 

In [ ]:
fig = px.line(x=df.index, y=df['word_count'])
fig.update_layout(xaxis_title="Speech",
    yaxis_title="N of 'war'")
fig.show()

In [ ]:
# james madison, 1813, war of 1812
print(df.loc['1813']['name'])
print(df.loc['1813']['freq_dist'].most_common(10))


# woodrow wilson, 1917, wwi
print(df.loc['1917']['name'])
print(df.loc['1917']['freq_dist'].most_common(10))

### Sentiment Analysis*

Sentiment analysis is an exploratory data analysis technique that "seeks to quantify the emotional intensity of words and phrases within a text." (quote from the [Programming Historian SA tutorial](https://programminghistorian.org/en/lessons/sentiment-analysis))

We can use more NLTK tools to run a simple sentiment analysis on our SOTU corpus. We'll download the vader_lexicon for sentiment analysis and the Vader and Sentiment modules. Don't worry if you see a warning that we don't have the twython library. We won't be using that since we're not analyzing twitter text.

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import sentiment
nltk.download('vader_lexicon')

Note that we could use a tokenizer that works best for sentiment analysis (see the commented out code below). Since we've already tokenized our text we'll stick with that corpus. 

In [ ]:
#nltk.download('punkt')
#tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

Let's initialize the vader SentimentIntensityAnalyzer and save it to a variable called sid.

In [ ]:
sid = SentimentIntensityAnalyzer()

Now we can use the features of the sentiment analysis tool. You can take a look at some of those features by typing sid. and then tabbing through the options.

In [ ]:
#sid

First let's look at the 'polarity_scores' for a specific speech. For Sentiment analysis we don't need the cleaned speech, so we'll go back to our original speeches list.

polarity_scores will give us positive and negative scores. This feature is built into VADER and can be requested on demand. We're going to go back to our original speeches dictionary, because we don't want to work with cleaned text for sentiment analysis. Punctuation, stop words, capitalization, etc. can all influence how positive or negative a text is deemed. For example, "AWESOME!" is more positive than "awesome?"

In [ ]:
scores = sid.polarity_scores(speeches['Roosevelt_1938'])
scores

Now let's look at the scores for the entire speeches corpus.
We'll create another dictionary, 'all_scores', that will use the speeches index as the key, and the polarity_scores output as its value. Note that this means that the value for each item in 'all_scores' will contain *another* dictionary.

This might take a few minutes to run because it has to analyze all 235 speeches.

In [ ]:
all_scores = {}
for k,v in speeches.items():
    all_scores[k] = sid.polarity_scores(v)

Now we can take a look at the scores for specific speeches by referencing the key of `all_scores`:

In [ ]:
all_scores['Trump_2017']

We can look at a specific score by referencing the key within the scores dictionary. 

In [ ]:
all_scores['Trump_2017']['neg']

From here, we can list all of the negative scores for the corpus. 

To keep it somewhat simple, let's just create a new dictionary that will only contain negative scores. We can create an empty dictionary called negative, then cycle through each key:value item in the all_scores dictionary from above. For each item, we'll assign the index number as its key and the negative score as its value.

In [ ]:
negative = {}
for score in all_scores.items():
    negative[score[0]] =  score[1]['neg']

This is a weird plot since our speeches dictionary, and our all_scores dictionaries are in alphabetical order, not in order by time. But we can still use it as a quick way to visualize the least and most negatives speeches. 

In [ ]:
x_speech = list(negative.keys())
y_neg = list(negative.values())
fig = px.line(x=x_speech, y=y_neg)
fig.update_layout(xaxis_title="Speech",
    yaxis_title="Negative value")
fig.show()

#### Most negative speeches
The graph gives us a nice visualization of some overall trends, and we can take a closer look at some of the most negative speeches here. We could also sort our dictionary, using the `sorted()` method, to list the speeches with the most negative scores in the corpus.

In [ ]:
sorted(negative, key=negative.get, reverse=True)[:5]

#### Least negative speeches
We can use the default sort (ascending values) to view the least negative speeches in the corpus.

In [ ]:
sorted(negative, key=negative.get)[:5]

## Where to go for help
- Contact LATISresearch@umn.edu
- [Libaries Text Mining guide](https://libguides.umn.edu/text-mining)
- [Join the Text as Data Practice Group email list]()